# 2. Importing Libraries
___
___

In [1]:
import cv2
import face_recognition
import sqlite3
import numpy as np
from datetime import datetime
import time
import pickle

# 2. Creating Database to try that code is valid
___
___

In [2]:
def connect_db():
    return sqlite3.connect('session.db', timeout=10)

def create_or_update_table():
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS attendance (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            NAME TEXT,
            entry_time TEXT,
            exit_time TEXT,
            encoding BLOB
        )
        ''')

        conn.commit()
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

 # 3.  This code logs a person's entry and exit times along with their face encoding in a database.
___
___

In [3]:
def When_entry(name, encoding):
    try:
        if isinstance(encoding, np.ndarray):
            print(f"Encoding to be stored for {name}: {encoding}")  # طباعة الـ encoding قبل التخزين
            encoding_data = pickle.dumps(encoding)  # ترميز الـ encoding باستخدام pickle
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            conn = connect_db()
            cursor = conn.cursor()
            cursor.execute("INSERT INTO attendance (NAME, entry_time, encoding) VALUES (?, ?, ?)", 
                           (name, current_time, sqlite3.Binary(encoding_data)))
            conn.commit()


            cursor.execute("SELECT encoding FROM attendance WHERE NAME = ?", (name,))
            stored_data = cursor.fetchone()

            if stored_data and stored_data[0] is not None:  
                try:
                    print(f"Stored encoding for {name}: {pickle.loads(stored_data[0])}")  
                except Exception as e:
                    print(f"Error decoding stored encoding for {name}: {e}")
            else:
                print(f"No encoding found or stored encoding is None for {name}")
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

def when_exit(name):
    try:
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("UPDATE attendance SET exit_time = ? WHERE NAME = ? AND exit_time IS NULL", 
                       (current_time, name))
        conn.commit()
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

### 4.The code finds and returns the name of a person whose face matches a given face encoding from the database.
___
___

In [4]:
def find_matching_face(face_encoding):
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("SELECT NAME, encoding FROM attendance")
        all_faces = cursor.fetchall()

        for stored_name, stored_encoding_blob in all_faces:
            if stored_encoding_blob: 
                try:
                    stored_encoding = pickle.loads(stored_encoding_blob) 

                    matches = face_recognition.compare_faces([stored_encoding], face_encoding)

                    if matches[0]: 
                        return stored_name
                except Exception as e:
                    print(f"Error decoding face encoding for {stored_name}: {e}")
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

    return None


# 4. Loading Haarcascade Model
___

In [5]:
face_detection = cv2.CascadeClassifier(r'c:\Users\skynet\Pictures\Ml Projects\haarcascade_frontalface_default.xml')

# 5. start the camera
___
___

In [6]:

camera = cv2.VideoCapture(0)

# 6.Track the presence status (in/out)
___

In [7]:
person_present = {}
last_seen_time = {}
cooldown_time = 20

# 7.Loop for Face Detection and Attendance Logging
___

In [8]:

create_or_update_table()

while True:
    st, frame = camera.read()
    if not st:
        print("Failed to capture image")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray_frame, 1.3, 5)
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y, x+w, y+h, x)])

        for face_encoding in face_encodings:
            name = find_matching_face(face_encoding)

            if name:
                cv2.putText(frame, f"{name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                if name not in person_present:
                    When_entry(name, face_encoding)
                    person_present[name] = True

                last_seen_time[name] = time.time()
            else:
                actual_name = input("Enter the name for the new person: ")
                When_entry(actual_name, face_encoding)
                person_present[actual_name] = True
                last_seen_time[actual_name] = time.time()
                cv2.putText(frame, f"{actual_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    for name in list(person_present.keys()):
        if time.time() - last_seen_time.get(name, 0) > cooldown_time:
            when_exit(name)
            del person_present[name]

    cv2.imshow('Camera', frame)

    if cv2.waitKey(33) & 0xff == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


Encoding to be stored for qqq: [-0.03708177  0.07392838  0.07168819 -0.06334274 -0.0502785  -0.03399943
 -0.06885545 -0.0468197   0.22074075 -0.12059987  0.20327982  0.0654911
 -0.26830825 -0.01459798  0.02076995  0.1507898  -0.1272534  -0.09589776
 -0.14275004 -0.12242798 -0.05797902  0.05658723  0.08457095 -0.02386622
 -0.16767967 -0.25502354 -0.11863537 -0.09209098  0.01331874 -0.11503343
  0.03730578  0.06263694 -0.17241286 -0.05682644  0.05556125  0.03741685
  0.01082759 -0.0394324   0.18618335  0.01723754 -0.1706215  -0.03472388
  0.01037508  0.25408164  0.21000403  0.00863452  0.0168989  -0.11342286
  0.10777859 -0.29981396  0.03397571  0.08918244  0.02185784  0.07146306
  0.07245173 -0.17593066  0.026777    0.1584563  -0.15978733  0.04412953
  0.04592987 -0.10248102 -0.08649217 -0.07414655  0.23044334  0.08508566
 -0.14592469 -0.20667808  0.18238693 -0.1298396  -0.01699467  0.11331223
 -0.11932048 -0.14988109 -0.30316919  0.03653984  0.41731653  0.13311814
 -0.13874479  0.07102